In [15]:
import requests
import os
import json
import jq
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="Hackathon TocToc")

api_key_toctoc = os.getenv('API_KEY_TOCTOC')
headers = {
    'Authorization': api_key_toctoc
}

communes = json.load(open('communes.json', 'r'))
regions = json.load(open('regions.json', 'r'))

In [3]:
def get_properties_sales(lat: float, long: float, radius: int,
                         page: int = 1, pageSize: int = 7, sortBy: str = 'publicationDate',
                         sortAsc: bool = True, idTypeFamily: int = 1, salableAreaMin = None, salableAreaMax = None) -> list:
    ''' Function that calls TOCTOC api asking for properties on sale around
    a location (lat, long), inside a radius and returns a list with the results.

    Args:
        lat (float): Latitude
        long (float): Longitude
        radius (int): Radius around lat,long

    Returns:
        list: A list containing the results of the search.
    '''
    payload = {}
    url = "https://gw.toctoc.com/1.0/info/sales?"
    params = {
        'lat': lat,
        'long': long,
        'radius': radius,
        'page': page,
        'pageSize': pageSize,
        'sortBy': sortBy,
        'sortAsc': sortAsc,
        'idTypeFamily': idTypeFamily,
    }
    response = requests.request("GET", url, headers = headers, data = payload, params = params)
    print(response.text)
    data = json.loads(response.text)
    return data["docs"]

# get_properties_sales test
get_properties_sales(-33.417044, -70.588743, 500)

{"status":"ok","statusCode":200,"message":"","docs":[{"_id":"6774f7978b2ddfa7e911a581","address":{"lat":-33.4187,"long":-70.5855,"communeName":"Las Condes","street":"Townhouse proyecto Net Zero en Málaga, El Golf  "},"area":{"balcony":0,"constructed":0,"land":231.4,"usable":163.4},"distance":353.2404633197919,"operationFamily":"Venta Usado","propertyFamilyType":"Casas","price":24490,"salableArea":163.4,"ufm2":149.87760097919215,"publicationDate":"2024-12-16T00:00:00.000Z"},{"_id":"6774f7978b2ddfa7e911a57f","address":{"lat":-33.4189,"long":-70.5867,"communeName":"Las Condes","street":"Moderno y vanguardista Townhouse en El Golf  "},"area":{"balcony":0,"constructed":0,"land":233.1,"usable":174.38},"distance":280.5717238235065,"operationFamily":"Venta Usado","propertyFamilyType":"Casas","price":24800,"salableArea":174.38,"ufm2":142.21814428260123,"publicationDate":"2024-12-16T00:00:00.000Z"},{"_id":"6774f7e38b2ddfa7e912027a","address":{"lat":-33.4156931,"long":-70.5852874,"communeName":"L

[{'_id': '6774f7978b2ddfa7e911a581',
  'address': {'lat': -33.4187,
   'long': -70.5855,
   'communeName': 'Las Condes',
   'street': 'Townhouse proyecto Net Zero en Málaga, El Golf  '},
  'area': {'balcony': 0, 'constructed': 0, 'land': 231.4, 'usable': 163.4},
  'distance': 353.2404633197919,
  'operationFamily': 'Venta Usado',
  'propertyFamilyType': 'Casas',
  'price': 24490,
  'salableArea': 163.4,
  'ufm2': 149.87760097919215,
  'publicationDate': '2024-12-16T00:00:00.000Z'},
 {'_id': '6774f7978b2ddfa7e911a57f',
  'address': {'lat': -33.4189,
   'long': -70.5867,
   'communeName': 'Las Condes',
   'street': 'Moderno y vanguardista Townhouse en El Golf  '},
  'area': {'balcony': 0, 'constructed': 0, 'land': 233.1, 'usable': 174.38},
  'distance': 280.5717238235065,
  'operationFamily': 'Venta Usado',
  'propertyFamilyType': 'Casas',
  'price': 24800,
  'salableArea': 174.38,
  'ufm2': 142.21814428260123,
  'publicationDate': '2024-12-16T00:00:00.000Z'},
 {'_id': '6774f7e38b2ddfa7e

In [ ]:
def get_region_id(regionName: str) -> int:
    '''Retrieves the id for a region from regions, searching for its name.
    The name must be one of the following:

    "Región de Arica y Parinacota"
    "Región de Tarapacá"
    "Región de Antofagasta"
    "Región de Atacama"
    "Región de Coquimbo"
    "Región de Valparaíso"
    "Región Metropolitana de Santiago"
    "Región del Libertador Gral. Bernardo O'Higgins"
    "Región del Maule"
    "Región de Ñuble"
    "Región del Biobío"
    "Región de La Araucanía"
    "Región de Los Ríos"
    "Región de Los Lagos"
    "Región Aysén del G. Carlos Ibáñez del Campo"
    "Región de Magallanes y de la Antártica Chilena"

    Args:
        regionName (str): Region's name.
    Returns:
        int: Region id.
    '''
    regions_name = ["Región de Arica y Parinacota", "Región de Tarapacá", "Región de Antofagasta",
"Región de Atacama", "Región de Coquimbo", "Región de Valparaíso", "Región Metropolitana de Santiago",
"Región del Libertador Gral. Bernardo O'Higgins", "Región del Maule", "Región de Ñuble", "Región del Biobío",
"Región de La Araucanía", "Región de Los Ríos", "Región de Los Lagos", "Región Aysén del G. Carlos Ibáñez del Campo",
"Región de Magallanes y de la Antártica Chilena"]

    if regionName not in regions_name:
        raise ValueError("Invalid regionName. Please check available names")
    else:
        return jq.compile(".data[] | select(.name == $regionName) | .id", args={"regionName": regionName}).input(regions).first()

# get region id test
get_region_id("Región Metropolitana de Santiago")

In [14]:
def get_city_lat_long(cityName: str, regionId: int) -> tuple:
    '''Searches the latitude and longitude for a given city and region
    to avoid duplicates.

    Args:
        cityName (str): City's name
        regionId (int): Region's id based (can be get from get_region_id)

    Returns:
        tuple: A tuple containing (lat, long)
    '''
    commune = jq.compile(".data[] | select(.name == $cityName, .region_id == $regionId) | .name", args={"cityName": cityName, "regionId": regionId}).input(communes).first()
    region = jq.compile(".data[] | select(.id == $regionId) | .name", args = {"regionId": regionId}).input(regions).first()
    location = geolocator.geocode(commune+", "+region+", Chile")
    return (location.latitude, location.longitude)

# test get city lat lnog
get_city_lat_long("Placilla", 6)

(-33.369172, -71.6680533)

In [16]:
def get_properties(city, region):
    region_id = get_region_id(region)
    lat, long = get_city_lat_long(city, region_id)
    return get_property_sales(lat,long, 500)
    